## Machine learning

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action = 'ignore')

df = pd.read_csv('Datasets/dataset_comp2.csv', sep = ';')
df

,corrente_cp11,corrente_cp12,corrente_cp13,corrente_cp31,vazaro_ar,pressao_ar,falha
0,96.900002,92.599998,97.000000,97.400002,21204.308594,516.897827,0
1,96.900002,92.500000,97.000000,97.400002,21073.390625,516.804749,0
2,96.900002,92.300003,96.900002,97.800003,21070.710938,517.828857,0
3,96.800003,92.500000,97.000000,97.400002,21106.867188,518.573669,0
4,97.000000,92.599998,97.199997,97.400002,21230.927734,518.759949,0
...,...,...,...,...,...,...,...
61357,99.699997,96.000000,111.000000,98.400002,20611.375000,592.868469,0
61358,99.500000,95.599998,110.699997,97.800003,20561.994141,594.916626,0
61359,99.599998,95.699997,110.900002,98.700005,20605.894531,594.637390,0
61360,99.400002,95.599998,110.699997,98.099998,20581.210938,595.568359,0


In [2]:
df['falha'].value_counts()

0    60183
1     1179
Name: falha, dtype: int64

### Separação entre dados de treino e teste

#### Aplicação do modelo de regressão linear

In [3]:
# A inteção da utilização deste modelo é avaliar os valoes da corrente do compressor 11 variando pressão e vazão

In [6]:
# Importando a biblioteca necesária para separação dos dados

from sklearn.model_selection import train_test_split

# Váriavel Y
y = df['corrente_cp11']

# Váriavel X
X = df[['vazaro_ar', 'pressao_ar']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 2811)

print("Dados separados em treino e teste")

Dados separados em treino e teste


In [7]:
# Modelo de regressão linear

from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [9]:
modelo = LinearRegression()
modelo.fit(X_train, y_train)
print("Modelo de treino criado")

Modelo de treino criado


#### Testando entrada de valores manuais

In [10]:
entrada = X_test[0:1]
entrada

,vazaro_ar,pressao_ar
12233,21298.654297,527.976929


In [17]:
corrente_pred = modelo.predict(entrada)
print('Valor de corrente para a entrada é : {}'.format((corrente_pred[0]).round(2)))

Valor de corrente para a entrada é : 96.78


In [31]:
vazaro_ar = 21038
pressao_ar = 513
entrada_test = [[vazaro_ar, pressao_ar]]
print("Valor previsto: {}".format(modelo.predict(entrada_test)[0]))

Valor previsto: 96.32492950285462


#### Aplicando o modelo com a variação de vazão e pressão

In [18]:
df.describe()

,corrente_cp11,corrente_cp12,corrente_cp13,corrente_cp31,vazaro_ar,pressao_ar,falha
count,61362.000000,61362.000000,61362.000000,61362.000000,61362.000000,61362.000000,61362.000000
mean,96.309420,91.236344,90.404480,96.356543,21095.299721,513.580014,0.019214
std,2.212627,8.045329,33.209106,1.924950,1137.053218,31.025603,0.137277
min,89.300003,-37.500000,-37.500000,90.599998,14367.592773,338.236633,0.000000
25%,94.599998,90.300003,94.500000,94.800003,21038.521484,504.329224,0.000000
50%,95.800003,91.500000,96.500000,96.099998,21354.267578,516.432373,0.000000
75%,98.300003,93.099998,100.800003,97.800003,21598.835938,526.859680,0.000000
max,102.699997,97.300003,114.400002,105.900002,23066.876953,637.836365,1.000000


In [20]:
import itertools

linhas = []

entrada = X_test[0:1]
entrada

for (i, j) in itertools.product(range(21038,23066), range(504,637)):
    vazaro_ar = i
    pressao_ar = j
    entrada = [[vazaro_ar, pressao_ar]]
    corrente_pred = int(modelo.predict(entrada)[0])
    
    linhas.append([corrente_pred, vazaro_ar, pressao_ar])
    
df_pred = pd.DataFrame(linhas, columns = ["corrente_cp11","vazao","pressao"])
print("Dataset concluído")

Dataset concluído


In [23]:
# Análise do menor valor de vazão e pressão para de corrente em alerta

df_alerta = df_pred[df_pred['corrente_cp11'] > 100]

In [26]:
df_alerta.nsmallest(1,'vazao')

,corrente_cp11,vazao,pressao
121,101,21038,625


In [27]:
df_alerta.nsmallest(1,'pressao')

,corrente_cp11,vazao,pressao
121,101,21038,625


##### Teremos alerta, sempre que a corrente for superior a 100 para valores de vazão menor o igual a 21038
##### Necessário adotar um outro modelo para entender o comportamento da pressão na corente